# Prepare the dataset and EDA

## combine the datasets

In [ ]:

import pandas as pd

df_features  = pd.read_json("D:\GITHUB\Summer-2025-ECE-597-Group11\data\output\dataset_v20250611.json")

In [9]:
import pandas as pd

df_with_no_feature  = pd.read_json("D:\GITHUB\Summer-2025-ECE-597-Group11\data\output\dataset_v20250618.json")

<>:3: SyntaxWarning: invalid escape sequence '\G'
<>:3: SyntaxWarning: invalid escape sequence '\G'
C:\Users\yck\AppData\Local\Temp\ipykernel_32080\1389715859.py:3: SyntaxWarning: invalid escape sequence '\G'
  df_with_no_feature  = pd.read_json("D:\GITHUB\Summer-2025-ECE-597-Group11\data\output\dataset_v20250618.json")


In [10]:
df_with_no_feature['label'].value_counts()

label
0    92695
1     2576
Name: count, dtype: int64

## Extracting meaningful features

In [ ]:
df_features.info()


## EDA

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

features = df_features.columns.tolist()[3:]

plt.figure(figsize=(25, 20))
for i, feature in enumerate(features):
    plt.subplot(4,4,i+1)
    plt.hist(df_features[feature], bins=20, edgecolor='k')
    plt.title(f"{feature} Distribution")


In [ ]:
numeric_features = ['spam_word_count', 'spam_word_ratio',"non_latin_count","non_latin_distinct_count","emoji_count","emoji_distinct_count"]

for feature in numeric_features:
    sns.boxplot(x='label', y=feature, data=df_features)
    plt.title(f"{feature} by Email Class")
    plt.show()


In [ ]:
df_features.info()

In [ ]:
binary_features = ['has_spam_word', 'has_link', 'has_email', 'has_number', 'has_dollar', 'has_emoji']

for feature in binary_features:
    sns.barplot(x='label', y=feature, data=df_features)
    plt.title(f"{feature} vs Spam Label")
    plt.ylabel(f"Mean Value of {feature}")
    plt.xlabel("Label (0 = Not Spam, 1 = Spam)")
    plt.show()


# Model and training

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

#
feature_cols = [col for col in df_features.columns if col not in ['subject', 'body', 'label']]

X = df_features[feature_cols]
y = df_features['label']

#
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

#
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

y_proba = model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, y_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))


In [ ]:
# Define the feature columns (should match training)
feature_cols = [col for col in df_features.columns if col not in ['subject', 'body', 'label']]

# Predict on new input


# Predict and view results
feature_cols


In [ ]:
df_features.columns

# Export the model weight.

In [ ]:
import os
import joblib

PATH_PKL = r'D:\GITHUB\Summer-2025-ECE-597-Group11\code\web_v2\models\parameters'
VERSION = "v2"

# Ensure directory exists
os.makedirs(PATH_PKL, exist_ok=True)

model_pkl = os.path.join(PATH_PKL, f'spam_classifier_model_{VERSION}.pkl')
scalar_pkl = os.path.join(PATH_PKL, f'spam_scaler_{VERSION}.pkl')
feature_txt = os.path.join(PATH_PKL, f'spam_feature_{VERSION}.txt')

joblib.dump(model, model_pkl)
joblib.dump(scaler, scalar_pkl)

with open(feature_txt, 'w') as f:
    for item in feature_cols:
        f.write(f"{item}\n")
